In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import torch

In [5]:
base_model_name = "mistralai/Mistral-7B-v0.1" 
adapter_path = "./mistral-indian-law-final"

In [6]:
peft_config = PeftConfig.from_pretrained(adapter_path)
print("PEFT Config:", peft_config)

PEFT Config: LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='mistralai/Mistral-7B-v0.1', revision=None, inference_mode=True, r=16, target_modules={'v_proj', 'q_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)


In [17]:
import gc

big_array = [i for i in range(10**7)]
del big_array
torch.cuda.empty_cache()
gc.collect()


0

In [18]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,              # Use 4-bit precision
    bnb_4bit_compute_dtype=torch.float16,  # Compute in FP16
    bnb_4bit_use_double_quant=True, # Double quantization for extra savings
    bnb_4bit_quant_type="nf4"       # NF4 quantization for better accuracy
)

In [19]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map="auto",              # Automatically distribute across GPU/CPU
    offload_folder="offload_dir",   # Directory for offloaded weights
    offload_state_dict=True,        # Offload state dict to disk
    low_cpu_mem_usage=True          # Minimize CPU memory usage
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:44<00:00, 22.17s/it]


In [20]:
model = PeftModel.from_pretrained(
    base_model,
    adapter_path,
    offload_folder="offload_dir"    # Offload adapters if needed
)

In [21]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_pro

In [22]:
tokenizer = AutoTokenizer.from_pretrained(adapter_path)

In [25]:
input_text = input("What do you want to ask: ")

In [31]:
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.7
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


what is the formal oath of secrecy for minister for india?
The formal oath of secrecy for Minister for India is:
"I, A.B., do swear in the name of God (or solemnly affirm) that I will not directly or indirectly communicate or reveal to any person or persons any matter which shall be brought under my consideration or shall become known to me as a Minister for India except as may be required for the due discharge of my duties as such Minister."  [Article 368(
